# Lab 4 

## Database Design 

### 3 tables:
    1.) Gene table: contains tuples of:
            {geneID, species, sequence}
    2.) Pathway descriptions: contains tuples of:
            {pathway name, pathway description}
    3.) Enzyme: contains tuples of:
            {enzyme name, geneID (, function)}

In [1]:
!ls

2019_9_29_Lab4.ipynb	 mus musculus_leptin.fa
fixedLargeFileError.txt  mus musculus_leptin.gb


In [2]:
import sqlite3
from Bio import Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import generic_protein
from Bio import SeqIO
import time

In [3]:
# Set up database
conn = sqlite3.connect("2019_9_29_Lab4_database.db")
c = conn.cursor()
c.execute("CREATE TABLE genes (geneId TEXT, type TEXT, species TEXT, sequence TEXT)")
conn.commit()
c.execute("CREATE TABLE enzyme (geneId TEXT, protein TEXT)")
conn.commit()

In [4]:
# Testing Entrez:
Entrez.email = 'jerry.yang@berkeley.edu'
# handle = Entrez.esearch(db='nucleotide',
#                        term='homo sapiens[ORGN] leptin',
#                        sort='relevance',
#                        idtype='acc',
#                        retmax = 1)

In [5]:
# for entry in Entrez.read(handle)['IdList']:
#     result = Entrez.efetch(db='nucleotide', id=entry, rettype='fasta', retmode='text')
    # print(result.read())
    
    
#     for seq_rec in SeqIO.parse("seqs.fa", "fasta"):   
#     seqs[seq_rec.id] = seq_rec

In [6]:
# Creating a helper function to run Entrez
def search(term, species='homo sapiens'):
    """ Function to search Entrez for a particular gene, given species and 
            search term
            
        Returns the top hit for that search as a dictionary containing the following information:
            {   "organism": [organism], 
                "sequence": [seq],
                "type": [rna or dna],
                "geneId": [geneID]
                }
            
        E.g.
        > search(species = "mus musculus", 'leptin')
        
        
        ">NM_008493.3 Mus musculus leptin (Lep), mRNA
        GAGGGATCCCTGCTCCAGCAGCTGCAAGGTGCAAGAAGAAGAAGATCCCAGGGAGGAAAATGTGCTGGAG ..."
        
        """
    handle = Entrez.esearch(db='nucleotide',
                   term='{species}[ORGN] {searchTerm}'.format(species = species, searchTerm = term),
                   sort='relevance',
                   idtype='acc',
                   retmax = 1)
    for entry in Entrez.read(handle)['IdList']:
        result = Entrez.efetch(db='nucleotide', id=entry, rettype='gb')
        # print(result.read())
        
        for rec in SeqIO.parse(result, "gb"):
            # print(rec.annotations)
            addToEnzymeDB(term, rec.annotations.get("accessions")[0], "enzyme")
            return {"organism": rec.annotations.get('organism'),
                    "geneID": rec.annotations.get("accessions")[0],
                    "type": rec.annotations.get("molecule_type"),
                    "sequence": str(rec.seq)
                    }
#             print(rec.annotations.get('organism'))
#             print(rec.seq)
    
#     def genOutputFastaName(species, term):
#         """ Helper function to systematically generate an output name"""
#         return "{sp}_{term}.gb".replace(" ", "_").format(sp = species, term = term)
    return {"organism": "N/A",
                    "geneID": "N/A",
                    "type": "N/A",
                    "sequence": "N/A"
                    }
#     with open(genOutputFastaName(species, term), "w") as output_handle:
#         gene_data = result.read()
        # output_handle.write(gene_data)
        
        # print(gene_data)
# search('leptin', species = 'mus musculus')

In [7]:
def addToGeneDB(gene_dict, geneDB):
    """ Adds the particular sequence to the gene database
    
    Assumes the following parameters for the gene DB:
    {geneID, species, type, sequence}"""

    c.execute("INSERT INTO genes (geneID, type, species, sequence) VALUES \
                ('{geneID}', '{t}', '{sp}', '{seq}')".format(geneID = gene_dict.get("geneID"),
                                                    t = gene_dict.get("type"),
                                                    sp = gene_dict.get("organism"),
                                                    seq = gene_dict.get("sequence")))
    conn.commit()
def addToEnzymeDB(protein, geneID, enzymeDB):
    c.execute("INSERT INTO enzyme (protein, geneID) VALUES \
                ('{prot}', '{geneID}')".format(prot = protein,
                                                    geneID = geneID,
                                                    ))
    conn.commit()

In [8]:
# addToGeneDB(search('leptin', species = 'mus musculus'), "genes")

In [9]:
# search('leptin', species = 'mus musculus')

In [10]:
# Input data:
genes = {'Glycolysis': ["alcohol dehydrogenase", 
                       "hexokinase", 
                        "triosephosphate isomerase", 
                       "glucokinase"],
        'TCA': ["phosphoenolpyruvate carboxykinase",
               "pyruvate carboxylase",
               "pyruvate synthase", # K00174
               "citrate synthase"],
         'Pentose Phosphate': # [E4.3.1.9, EC 4.2.1.39   ]
             ["glucosaminate ammonia-lyase", 
               "2-dehydro-3-deoxy-phosphogluconate", 
              "gluconate dehydratase",
              "glucose oxidase"]
         
        }
species = ['Homo sapien', 'Drosophila melanogaster', 'Escherichia coli']
# print("\n".join(genes['Glycolysis']))

In [11]:
# Helper function: visualizes the elements in the database
def sql_fetch(dbName, con = conn):
 
    cursorObj = con.cursor()
 
    cursorObj.execute('SELECT * FROM {name}'.format(name=dbName))
 
    rows = cursorObj.fetchall()
 
    for row in rows:
 
        print(row)
# sql_fetch(conn)

In [12]:
# Generate the pathway database
# def createTable(tableName, fieldsList, typesList):
#     pairs = zip(fieldsList, typesList)
#     c.execute("CREATE TABLE {t} (geneId TEXT, type TEXT, species TEXT, sequence TEXT)")
#     conn.commit()
c.execute("CREATE TABLE pathway (pathwayName TEXT, protein TEXT)")
conn.commit()

In [13]:
def addToPathwayDB(pathway, protein, geneDB):
    """ Adds the particular sequence to the pathway database
    
    Assumes the following parameters for the gene DB:
    {geneID, species, type, sequence}"""

    c.execute("INSERT INTO pathway (pathwayName, protein) VALUES \
                ('{path}', '{prot}')".format(prot = protein,
                                                    path = pathway,
                                                    ))
    conn.commit()

for pathway in genes.keys():
    for protein in genes.get(pathway):
        addToPathwayDB(pathway, protein ,"pathway")
        


In [14]:
sql_fetch("pathway")

('Glycolysis', 'alcohol dehydrogenase')
('Glycolysis', 'hexokinase')
('Glycolysis', 'triosephosphate isomerase')
('Glycolysis', 'glucokinase')
('TCA', 'phosphoenolpyruvate carboxykinase')
('TCA', 'pyruvate carboxylase')
('TCA', 'pyruvate synthase')
('TCA', 'citrate synthase')
('Pentose Phosphate', 'glucosaminate ammonia-lyase')
('Pentose Phosphate', '2-dehydro-3-deoxy-phosphogluconate')
('Pentose Phosphate', 'gluconate dehydratase')
('Pentose Phosphate', 'glucose oxidase')


In [15]:
# Now, generate the gene database via ENTREZ:
for sp in species:
    for key in genes.keys():
        for g in genes.get(key):
            result = search(g, species=sp)
            # print(result)
            addToGeneDB(result, "genes")
            print(sp, g)
            time.sleep(5) # Adding sleep to avoid http "Too Many requests error"

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
sql_fetch("genes")